In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [2]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = user_secrets.get_secret("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "langchain kaggle"
os.environ["MISTRAL_API_KEY"] = user_secrets.get_secret("mistral_legalBot")

In [3]:
!pip install -U langchain-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2

In [5]:
from langchain.chat_models import init_chat_model

model = init_chat_model("mistral-large-latest", model_provider="mistralai")

# Pydantic Model

In [6]:
from pydantic import BaseModel, Field
from typing import List

class DailyPlan(BaseModel):
    day: int = Field(description="Day number of the trip")
    activities: List[str] = Field(description="List of activities for the day")

class Itinerary(BaseModel):
    destination: str = Field(description="The city or place to visit")
    days: int = Field(description="Number of days for the trip")
    preferences: List[str] = Field(description="List of user preferences (e.g., food, history, art)")
    itinerary: List[DailyPlan] = Field(description="List of daily plans with activities")

# Ouput Parsing

In [15]:
model_with_structure = model.with_structured_output(Itinerary)

In [16]:
user_input = "I want to visit Paris for 3 days. I love food, history, and art."

In [17]:
structured_output = model_with_structure.invoke(user_input)

In [18]:
print(type(structured_output))
print(structured_output)

<class '__main__.Itinerary'>
destination='Paris' days=3 preferences=['food', 'history', 'art'] itinerary=[]


# With Prompt Eng.

In [7]:
from langchain.output_parsers import PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=Itinerary)
format_instructions = output_parser.get_format_instructions()

In [8]:
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"DailyPlan": {"properties": {"day": {"description": "Day number of the trip", "title": "Day", "type": "integer"}, "activities": {"description": "List of activities for the day", "items": {"type": "string"}, "title": "Activities", "type": "array"}}, "required": ["day", "activities"], "title": "DailyPlan", "type": "object"}}, "properties": {"destination": {"description": "The city or place to visit", "title": "Destination", "type": "string"}, "days": {"description": "Number of days for the trip", "title": "Days", "type": "integer"}, "

In [11]:
from langchain.prompts import ChatPromptTemplate
prompt_template = """\
You are a travel itinerary planner. Plan the travel itinerary from user input. Make appropriate assumption if some information is missing.\

user: {user_input}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=prompt_template)

messages = prompt.format_messages(user_input=user_input, 
                                format_instructions=format_instructions)

print(messages)

[HumanMessage(content='You are a travel itinerary planner. Plan the travel itinerary from user input. Make appropriate assumption if some information is missing.\nuser: I want to visit Paris for 3 days. I love food, history, and art.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"DailyPlan": {"properties": {"day": {"description": "Day number of the trip", "title": "Day", "type": "integer"}, "activities": {"description": "List of activities for the day", "items": {"type": "string"}, "title": "Activities", "type": "array"}}, "required": ["day", "activities"], "title": "

In [13]:
response = model.invoke(messages)

In [14]:
itinerary = output_parser.parse(response.content)
print(itinerary)

destination='Paris' days=3 preferences=['food', 'history', 'art'] itinerary=[DailyPlan(day=1, activities=['Visit the Louvre Museum', 'Have lunch at a local café', 'Explore the Marais district', 'Dinner at a Michelin-starred restaurant']), DailyPlan(day=2, activities=['Tour the Palace of Versailles', 'Lunch at a traditional French bistro', "Visit the Musée d'Orsay", 'Evening stroll along the Seine River']), DailyPlan(day=3, activities=['Visit Notre-Dame Cathedral', 'Lunch at a French bakery', 'Walk through the Montmartre neighborhood', 'Enjoy a French wine tasting'])]
